## Day 20: Trench Map

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2021/tree/HEAD/notebooks/day20/puzzle.ipynb)

[Problem statement](https://adventofcode.com/2021/day/20)

In [ ]:
module Bounds = 
    type T = 
        {MinX: int; MaxX: int; MinY: int; MaxY: int} 

    let contains x y bounds = 
        bounds.MinX <= x && x <= bounds.MaxX && bounds.MinY <= y && y <= bounds.MaxY

    let extend x y bounds = 
        {
            MinX = bounds.MinX - x
            MaxX = bounds.MaxX + x
            MinY = bounds.MinY - y
            MaxY = bounds.MaxY + y
        }
    let map f bounds = 
        seq {
            for x in bounds.MinX..bounds.MaxX do
            for y in bounds.MinY..bounds.MaxY do
                yield f x y 
        }
    let allIndices bounds = 
        map (fun x y -> x,y) bounds

type Input = {Bounds: Bounds.T; LightPixels: Set<int*int>; Enhancements: bool[]}
type State = {Bounds: Bounds.T; LightPixels: Set<int*int>; Enhancements: bool[]; InfinityValueRotation: bool list;}

In [ ]:
#load "../common.fsx"


let parse input = 
    let lines = readLines input |> Seq.toArray
    let enhancements = lines.[0]
    let length = lines.[1..] |> Seq.head |> String.length
    let height = Array.length lines
    let g = lines.[1..] |> Seq.mapi (fun i line -> line |> Seq.mapi (fun j c -> i,j,c)) |> Seq.collect id

    {
        Bounds = { MinX = 0; MaxX = length - 1 ; MinY = 0; MaxY = height - 1 } 
        LightPixels = g |> Seq.choose (function | (i,j,'#') -> Some (i,j) | _ -> None) |> Set.ofSeq
        Enhancements = enhancements |> Seq.map ((=)'#') |> Seq.toArray
    }




In [ ]:
#!value --name sampleRaw --from-file ./sample

In [ ]:
#!value --name inputRaw --from-file ./data

In [ ]:
#!share sampleRaw --from value
#!share inputRaw --from value

let enrich infinityValueRotation (input: Input) = 
    {
        Bounds = input.Bounds
        LightPixels = input.LightPixels
        Enhancements = input.Enhancements
        InfinityValueRotation = infinityValueRotation
    }

let sampleData = parse sampleRaw |> enrich [false] |> displayPipe
let actualData = parse inputRaw |> enrich [false; true] |> displayPipe

Bounds,LightPixels,Enhancements,InfinityValueRotation
"{ { MinX = 0 MaxX = 4 MinY = 0 MaxY = 5 }: MinX: 0, MaxX: 4, MinY: 0, MaxY: 5 }","[ ( 0, 0 ), ( 0, 3 ), ( 1, 0 ), ( 2, 0 ), ( 2, 1 ), ( 2, 4 ), ( 3, 2 ), ( 4, 2 ), ( 4, 3 ), ( 4, 4 ) ]","[ False, False, True, False, True, False, False, True, True, True, True, True, False, True, False, True, False, True, False, True ... (492 more) ]",[ False ]


Bounds,LightPixels,Enhancements,InfinityValueRotation
"{ { MinX = 0 MaxX = 99 MinY = 0 MaxY = 100 }: MinX: 0, MaxX: 99, MinY: 0, MaxY: 100 }","[ ( 0, 7 ), ( 0, 20 ), ( 0, 22 ), ( 0, 23 ), ( 0, 27 ), ( 0, 28 ), ( 0, 31 ), ( 0, 35 ), ( 0, 36 ), ( 0, 38 ), ( 0, 41 ), ( 0, 42 ), ( 0, 43 ), ( 0, 44 ), ( 0, 46 ), ( 0, 49 ), ( 0, 51 ), ( 0, 52 ), ( 0, 55 ), ( 0, 57 ) ... (4998 more) ]","[ True, True, True, False, True, False, False, False, False, True, False, False, False, False, False, True, True, False, False, False ... (492 more) ]","[ False, True ]"


In [ ]:
let rec steps count (state : State) = 
    if count <= 0 then state
    else 
        let newBounds = state.Bounds |> Bounds.extend 1 1 
        let allExtendedIndices = newBounds |> Bounds.allIndices
        let infPixel::infPixelsRest = state.InfinityValueRotation
        let newPixels = 
            seq {
                for (i, j) in allExtendedIndices do 
                    let adjs = [
                        for adji in [i+1; i; i-1] do
                        for adjj in [j+1; j; j-1] do
                            yield adji, adjj
                    ]
                    let newPixelIndex = 
                        adjs
                        |> List.mapi
                            (
                                fun pos (adji, adjj) ->   
                                    if (Bounds.contains adji adjj state.Bounds)
                                    then 
                                        if (state.LightPixels |> Set.contains (adji,adjj))
                                        then 
                                            1 <<< pos
                                        else 0
                                    else if infPixel then 1 <<< pos else 0
                            )
                        |> List.sum
                    if state.Enhancements.[newPixelIndex] 
                    then yield (i,j)
            }
            |> Set.ofSeq
        steps (count - 1) {state with LightPixels = newPixels; Bounds = newBounds; InfinityValueRotation = infPixelsRest @ [infPixel]}

In [ ]:
steps 2 sampleData |> (fun x -> x.LightPixels) |> Set.count |> display
steps 2 actualData |> (fun x -> x.LightPixels) |> Set.count |> display

35

5065

In [ ]:
steps 50 sampleData |> (fun x -> x.LightPixels) |> Set.count |> display
steps 50 actualData |> (fun x -> x.LightPixels) |> Set.count |> display

3351

14790